In [1]:
from gymjsp.jsspenv import HeuristicJsspEnv
from tianshou.env import SubprocVectorEnv
import numpy as np
import random
import torch


seed = 0
def seed_torch(seed):
    torch.manual_seed(seed)
    if torch.backends.cudnn.enabled:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
np.random.seed(seed)
random.seed(seed)
seed_torch(seed)

num_train_envs = 20
def helper_function():
  env = HeuristicJsspEnv("ft06")
  return env

env = HeuristicJsspEnv("ft06")
train_envs = SubprocVectorEnv([helper_function for _ in range(num_train_envs)])
test_envs = SubprocVectorEnv([helper_function for _ in range(100)])

In [6]:
import torch, numpy as np
from torch import nn
from tianshou.utils.net.common import ActorCritic, Net
from rl.network import Dueling_Network

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
state_shape = state_shape[0]

net = Net(state_shape, action_shape, hidden_sizes=[64,64], device=device, dueling_param=({"hidden_sizes":[64,64]}, {"hidden_sizes":[64,64]}))

optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [7]:
import tianshou as ts
from tianshou.policy import DQNPolicy, RainbowPolicy

policy = ts.policy.DQNPolicy(net, optim, discount_factor=0.99, estimation_step=1, target_update_freq=320)

In [8]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 20), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

In [9]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=1, step_per_epoch=10000, step_per_collect=2000,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05))
    
print(f'Finished training! Use {result["duration"]}')

Epoch #1:  20%|##        | 2000/10000 [05:05<20:20,  6.56it/s] 


KeyboardInterrupt: 

In [ ]:
from tianshou.data import Batch

obs = env.reset()
done = False
score = 0

while not done:
    action = policy(Batch(obs=obs[np.newaxis, :], info=None)).act.item()
    next_state, reward, done, info = env.step(action)

    obs = next_state
    score += reward

In [ ]:
info

{'cost': 0, 'wait': 0, 'total action': 69, 'makespan': 69}